# AlgoBulls Assignment

### Import required packages

In [201]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [25]:
import requests
class ScriptData:
    
    def __init__(self):
        self.info1 = {}
        self.data1 = {}
        
    def __getitem__(self, script):
        return self.data1[script]
    
    def __setitem__(self, script, df):
        self.data1[script] = df
        
    def __contains__(self, script):
        return script in self.data1.keys()
    
    def fetch_intraday_data(self,script):
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol='+script+'&interval=60min&apikey=1912UMDVJ3T55DBL'
        r = requests.get(url)
        data = r.json()
        self.info1[script] = data['Time Series (60min)']
        
    def convert_intraday_data(self,script):
        df = pd.DataFrame(self.info1[script].values())
        df.columns = ['open', 'high', 'low', 'close', 'volume']
        df['timestamp'] = self.info1[script].keys()
        df = df[['timestamp','open','high','low','close','volume']]
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)
        df['volume'] = df['volume'].astype(int)
        self.__setitem__(script, df)


In [26]:
script_data = ScriptData()

In [27]:
script_data.fetch_intraday_data('AAPL')

In [28]:
script_data.convert_intraday_data('AAPL')

In [119]:
script_data['AAPL']

In [30]:
'AAPL' in script_data

True

In [85]:
script_data.fetch_intraday_data('GOOGL')

In [86]:
script_data.convert_intraday_data('GOOGL')

In [87]:
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-06-14 19:00:00,123.6800,123.7600,123.55,123.5500,7106
1,2023-06-14 18:00:00,123.8882,123.8882,123.71,123.7100,8857
2,2023-06-14 17:00:00,123.6700,123.8876,123.67,123.8876,204798
3,2023-06-14 16:00:00,123.6700,123.8942,123.57,123.8600,1701141
4,2023-06-14 15:00:00,123.2400,124.0200,122.88,123.7000,4835358
...,...,...,...,...,...,...
95,2023-06-07 04:00:00,127.1800,127.1800,126.82,126.8800,3495
96,2023-06-06 19:00:00,127.0400,127.2500,127.00,127.2500,15358
97,2023-06-06 18:00:00,127.3100,127.3100,127.10,127.2000,9130
98,2023-06-06 17:00:00,127.4500,127.9900,127.31,127.3600,6505


In [174]:
'GOOGL' in script_data

True

In [32]:
'NVDA' in script_data

False

In [175]:
def indicator1(df, timeperiod):
    list1 = []
    for i in range(len(df)+1):
        if i < timeperiod - 1:
            list1.append(None)
        elif i >= timeperiod:
            sum = 0
            for j in range(i-timeperiod, i):
                sum += df['close'][j]
            mean = sum/timeperiod
            list1.append(mean)
    df2 = pd.DataFrame()
    df2['timestamp'] = df['timestamp']
    df2['indicator'] = list1
    return df2

In [118]:
indicator1(script_data['AAPL'], 5)

,timestamp,indicator
0,2023-06-14 19:00:00,NaN
1,2023-06-14 18:00:00,NaN
2,2023-06-14 17:00:00,NaN
3,2023-06-14 16:00:00,NaN
4,2023-06-14 15:00:00,184.000
5,2023-06-14 14:00:00,183.936
6,2023-06-14 13:00:00,183.872
7,2023-06-14 12:00:00,183.880
8,2023-06-14 11:00:00,183.882
9,2023-06-14 10:00:00,183.846


In [176]:
indicator1(script_data['GOOGL'], 5)

,timestamp,indicator
0,2023-06-14 19:00:00,NaN
1,2023-06-14 18:00:00,NaN
2,2023-06-14 17:00:00,NaN
3,2023-06-14 16:00:00,NaN
4,2023-06-14 15:00:00,123.74152
...,...,...
95,2023-06-07 04:00:00,127.00600
96,2023-06-06 19:00:00,127.02600
97,2023-06-06 18:00:00,127.04600
98,2023-06-06 17:00:00,127.12800


In [197]:
class Strategy:
    
    def __init__(self, script):
        self.script = script
        
    def get_script_data(self):
        self.df_close = script_data[self.script]
        self.df_indi = indicator1(self.df_close, 5)
        self.na = self.df_indi['indicator'].isna().sum()
        
    def get_signals(self):
        
        df_signals = pd.DataFrame()
        df_signals['timestamp'] = self.df_close['timestamp'][self.na:]
        df_signals['signal'] = 'NO_SIGNAL'
        
        if self.df_indi['indicator'][self.na] > self.df_close['close'][self.na]:
            flag = True
        else:
            flag = False
            
        for x in range(self.na, len(self.df_indi)):
            if flag == False:
                if self.df_indi['indicator'][x] > self.df_close['close'][x]:
                    df_signals['signal'][x] = 'BUY'
                    flag = True
            elif flag == True:
                if self.df_indi['indicator'][x] < self.df_close['close'][x]:
                    df_signals['signal'][x] = 'SELL'
                    flag = False
        
        df_signals = df_signals[df_signals['signal'] != 'NO_SIGNAL']
        
        return df_signals
            

In [198]:
strategy = Strategy('GOOGL')

In [199]:
strategy.get_script_data()

In [202]:
strategy.get_signals()

,timestamp,signal
7,2023-06-14 12:00:00,SELL
10,2023-06-14 09:00:00,BUY
12,2023-06-14 07:00:00,SELL
19,2023-06-13 16:00:00,BUY
22,2023-06-13 13:00:00,SELL
28,2023-06-13 07:00:00,BUY
30,2023-06-13 05:00:00,SELL
32,2023-06-12 19:00:00,BUY
43,2023-06-12 08:00:00,SELL
45,2023-06-12 06:00:00,BUY
